In [4]:
!pip -q install -U langchain langchain-core langchain-openai openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 3.2 MB/s eta 0:00:00


In [ ]:
import os
api_key = os.getenv("GITHUB_MODELS_KEY") 


In [2]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Add two integers."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

TOOLS = [add, multiply]


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=api_key,
    base_url="https://models.github.ai/inference",
)


In [5]:
import json
from langchain_core.messages import HumanMessage, ToolMessage

llm_with_tools = llm.bind_tools(TOOLS)

def run_tool_calling(question: str):
    messages = [HumanMessage(content=question)]
    
    # 1) Ask the model
    ai_msg = llm_with_tools.invoke(messages)
    messages.append(ai_msg)

    # 2) If it requested tools, execute them
    tool_calls = getattr(ai_msg, "tool_calls", None) or []
    for call in tool_calls:
        name = call["name"]
        args = call.get("args", {}) or {}
        tool_id = call["id"]

        tool_map = {t.name: t for t in TOOLS}
        if name not in tool_map:
            raise ValueError(f"Tool not found: {name}")

        result = tool_map[name].invoke(args)

        # Send tool result back to model
        messages.append(ToolMessage(content=str(result), tool_call_id=tool_id))

    # 3) Ask the model again to produce the final answer (if tools were used)
    final = llm_with_tools.invoke(messages)
    return final.content

print(run_tool_calling("What is (7 + 11) * 3 ? Use tools."))


The result of \( (7 + 11) * 3 \) is 54.
